In [1]:
%load_ext sql
%sql sqlite://

'Connected: None@None'

Football Season
===============

We will translate the Football season ER diagram we constructed in class into a database instance. In general, we traslate each entity set and relationship into tables. The entity sets have primary keys that are noted in the ER diagram. Relationships inherit the primary keys of the entity sets they connect. Their primary key is the composition of primary keys from the connecting entity sets, each of which is also a foreign key in the relationship table.

### Strong entity sets
First, translate strong entity sets to tables:
> * Create a table for the entity set.
> * Make each attribute of the entity set a field of the table, giving it an appropriate type.
> * Set the primary key the same as the set of key attributes in the entity set.

In [2]:
# Games table
%sql DROP TABLE IF EXISTS Games;
%sql CREATE TABLE Games ( date DATE, location VARCHAR(40), PRIMARY KEY (date, location) );
# Cities table
%sql DROP TABLE IF EXISTS Cities;
%sql CREATE TABLE Cities ( name VARCHAR(40) PRIMARY KEY );
# Position table
%sql DROP TABLE IF EXISTS Position;
%sql CREATE TABLE Position ( ptype VARCHAR(20) PRIMARY KEY );
# Players table
%sql DROP TABLE IF EXISTS Players;
%sql CREATE TABLE Players ( player_id UNSIGNED INT PRIMARY KEY, name VARCHAR(50) );
# Plays table
%sql DROP TABLE IF EXISTS Plays;
%sql CREATE TABLE Plays ( play_id UNSIGNED INT PRIMARY KEY, yardage_diff INT, touchdown BOOL );

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

### Weak entity sets
Weak entity sets use the primary key of the owning relation as part of their primary key. Note that the relationship connecting the weak entity set to its owning entity set is superfluous and will not be traslated into a table. Follow this procedure to translate weak entity sets into tables:
> * Create a table for the weak entity set.
> * Make each attribute of the weak entity set a field of the table.
> * Add fields for the primary key attributes of the owning entity set.
> * Declare a foreign key constraint on these identifying owner fields.
> * Optionally, set a constraint to automatically delete any tuples in the table for which there are no owners.


In [3]:
%%sql 
/* Teams table */
DROP TABLE IF EXISTS Teams;
CREATE TABLE Teams ( 
    city_name VARCHAR(40), 
    team_name VARCHAR(50), 
    PRIMARY KEY (city_name, team_name), 
    FOREIGN KEY (city_name) REFERENCES Cities
        ON DELETE CASCADE
);

/* Weight table */
DROP TABLE IF EXISTS Weight;
CREATE TABLE Weight ( 
    player_id UNSIGNED INTEGER, 
    time_period INTEGER,
    weight FLOAT,
    PRIMARY KEY (player_id, time_period), 
    FOREIGN KEY (player_id) REFERENCES Players
        ON DELETE CASCADE
);


Done.
Done.
Done.
Done.


[]

### Subclasses
Subclasses also receive their own tables:
> * For each subclass, declare a table using superclass’s primary key and the subclass’s extra attributes.
> * Declare the primary key from the superclass as the primary key of the subclass
> * Add a foreign key constraint to the primary key of the superclass.
> * Optionally, set a constraint to automatically delete any tuples in the table for which there are no superclass records.

In [4]:
%%sql
/* OffensePosition */
DROP TABLE IF EXISTS OffensePosition;
CREATE TABLE OffensePosition ( 
    ptype VARCHAR(20) PRIMARY KEY,
    offensive_postion_type CHAR(2),
    FOREIGN KEY (ptype) REFERENCES Position
        ON DELETE CASCADE
);


/* DefensePosition */
DROP TABLE IF EXISTS DefensePosition;
CREATE TABLE DefensePosition ( 
    ptype VARCHAR(20) PRIMARY KEY,
    defensive_postion_type CHAR(2),
    FOREIGN KEY (ptype) REFERENCES Position
        ON DELETE CASCADE
);

Done.
Done.
Done.
Done.


[]

In the case that a `Position` could be both an `OffensePosition` and a `DefensePosition` at the same time (makes so sense in Football, though...), we could create a third table to capture the intersection of these subclasses. What would be the alternative for capturing these intersection entities without creating this third table?

In [5]:
%%sql
/* OffenseDefensePosition */
DROP TABLE IF EXISTS OffenseDefensePosition;
CREATE TABLE OffenseDefensePosition ( 
    ptype VARCHAR(20) PRIMARY KEY,
    offensive_postion_type CHAR(2),
    defensive_postion_type CHAR(2),
    FOREIGN KEY (ptype) REFERENCES Position
        ON DELETE CASCADE
);

Done.
Done.


[]

### Many-to-many relations
Create a table for each relationship, except those connecting a weak entity set to its owner entity set.
> * Create a table for the relationship set.
> * Add all primary keys of the participating entity sets as fields in the table.
> * Add a field for each attribute of the relationship.
> * Declare a primary key using all key fields from the participating entity sets.
> * Declare foreign key constraints for all key fields from their respective entity sets.
> * Optionally, set a constraint to automatically delete any tuples in the table for which entities no longer exist.

In [6]:
%%sql
/* PRIn */
DROP TABLE IF EXISTS PRIn;
CREATE TABLE PRIn ( 
    player_id INTEGER,
    play_id INTEGER,
    PRIMARY KEY (player_id, play_id),
    FOREIGN KEY (player_id) REFERENCES Players
        ON DELETE CASCADE,
    FOREIGN KEY (play_id) REFERENCES Plays
        ON DELETE CASCADE
);


/* ParticipateIn */
DROP TABLE IF EXISTS ParticipateIn;
CREATE TABLE ParticipateIn ( 
    player_id INTEGER,
    play_id INTEGER,
    participation_type VARCHAR(30),
    PRIMARY KEY (player_id, play_id),
    FOREIGN KEY (player_id) REFERENCES Players
        ON DELETE CASCADE,
    FOREIGN KEY (play_id) REFERENCES Plays
        ON DELETE CASCADE
);

Done.
Done.
Done.
Done.


[]

### Many-to-one relations
We could follow the same strategy as above for many-to-one and one-to-one relationships. However, one problem with this strategy is that we cannot capture participation constraints for these relations. For example, given the relation table `PlaysPosition` below, there is nothing to prevent us from signing up a player with multiple teams at the same time. 

In [7]:
%%sql
/* PlaysPosition */
DROP TABLE IF EXISTS PlaysPosition;
CREATE TABLE PlaysPosition ( 
    ptype VARCHAR(20),
    player_id INTEGER,
    PRIMARY KEY (ptype, player_id),
    FOREIGN KEY (ptype) REFERENCES Position
        ON DELETE CASCADE,
    FOREIGN KEY (player_id) REFERENCES Players
        ON DELETE CASCADE
);

Done.
Done.


[]

Instead, since the `PlaysPosition` relation is many-to-one, we don't in fact need a table for the relation itself. We can instead register the `Position type` directly in the `Players` table.

Given a many-to-one relationship, where the many entity set is the `source` and the one entity set is the `target`,
> * Add every primary key field of the target as a field in the source.
> * Declare these fields as foreign keys.
> * Declare these fields as not null. (This enforces the participation constraint)

Instead of dropping and recreating the `Players` table, we can use an `ALTER` statement to change the fields and constraints in the table. Take a moment to think about altering the table. What if the `Players` table already had records? How can we make sure the added `ptype` field is all populated, so we do not violate the NOT NULL participation constraint?

In [8]:
%%sql
DROP TABLE IF EXISTS PlaysPosition;
ALTER TABLE Players ADD COLUMN ptype VARCHAR(20);
ALTER TABLE Players ADD FOREIGN KEY (ptype) REFERENCES Position;

Done.
Done.
(sqlite3.OperationalError) near "FOREIGN": syntax error [SQL: u'ALTER TABLE Players ADD FOREIGN KEY (ptype) REFERENCES Position;']


However, SQLite does not support adding key constraints in `ALTER TABLE` commands. Therefore, we'll just re-create the the `Players` table.


In [9]:
%%sql
DROP TABLE IF EXISTS Players;
CREATE TABLE Players ( 
    player_id UNSIGNED INT PRIMARY KEY, 
    name VARCHAR(50),
    ptype VARCHAR(20) NOT NULL,
    FOREIGN KEY (ptype) REFERENCES Position
);

Done.
Done.


[]

### One-to-one relationships

For one-to-one relationships, we follow a similar strategy as in the many-to-one case, except each side can be considered `source` and `target` (follow the arrows).
> * Add every primary key field of one entity set as a field in the other entity set.
> * Declare these fields as foreign keys.
> * Declare these fields as not null. (This enforces the participation constraint)

We have no one-to-one examples in our E/R diagram.

## Exercise

Translate the remaining relations into tables. Pay close attention to the dual relationship between the `Games` and `Teams` entity sets. Also, note that the arrow from `Players` to `Teams` is not thick (i.e., participation is partial, not total -- hint: can the associated team key be NULL?).

In [10]:
%%sql
/* PlaysPosition relation */
DROP TABLE IF EXISTS Players;
CREATE TABLE Players ( 
    player_id UNSIGNED INT PRIMARY KEY, 
    name VARCHAR(50),
    ptype VARCHAR(20) NOT NULL,
    city_name VARCHAR(40),
    team_name VARCHAR(50),
    FOREIGN KEY (ptype) REFERENCES Position
        ON DELETE CASCADE,
    FOREIGN KEY (team_name, city_name) REFERENCES Teams
        ON DELETE CASCADE
);

/* PlayingAway and PlayingHome relations */
DROP TABLE IF EXISTS Games;
CREATE TABLE Games ( 
    date DATE, 
    location VARCHAR(40), 
    home_city_name VARCHAR(40) NOT NULL,
    home_team_name VARCHAR(50) NOT NULL,
    away_city_name VARCHAR(40) NOT NULL,
    away_team_name VARCHAR(50) NOT NULL,
    PRIMARY KEY (date, location),
    FOREIGN KEY (home_team_name, home_city_name) REFERENCES Teams
        ON DELETE CASCADE,    
    FOREIGN KEY (away_team_name, away_city_name) REFERENCES Teams
        ON DELETE CASCADE
);

/* OccurIn relation */
DROP TABLE IF EXISTS Plays;
CREATE TABLE Plays ( 
    play_id UNSIGNED INT PRIMARY KEY, 
    yardage_diff INT, 
    touchdown BOOL,
    date date NOT NULL,
    location VARCHAR(40) NOT NULL,
    FOREIGN KEY (date, location) REFERENCES Games
        ON DELETE CASCADE
);

Done.
Done.
Done.
Done.
Done.
Done.


[]